In [72]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import re
import time
from tqdm import tqdm

In [73]:
# def best_weights(feature_df, results_df):
#     num_features = len(feature_df.columns[2:-1])
#     num_iterations = 2000
#     weights=[]
#     accuracy_list = []
#     weight_list = []
#     weights_df = pd.DataFrame()
#     results_range=results_df[results_df['Year'].isin(feature_df['Year'].unique())]
#     # performance_dict={}
#     # print(results_range)
#     i=0
#     for single_weight in tqdm(range(num_iterations)):
#         results_index=[]
#         temp_df=pd.DataFrame()
#         comparison_column=[]
#         weights = np.random.randint(low=-1, high=1, size=num_features).astype(np.float32)
#         # weights = np.random.random(num_features)
#         # weights /= np.sum(weights)
#         fran_weights = feature_df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
#         fran_weights = fran_weights.T
#         fran_weights = fran_weights.multiply(feature_df['Strength of Schedule']*weights[-2], axis=0).multiply(feature_df['Last 10 Rating']*weights[-1], axis=0)
#         # print(fran_weights)
#         # fran_weights = fran_weights.drop(['Strength of Schedule', 'Last 10 Rating'], axis=1)
#         fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
#         fran_weights.insert(0,"Team",feature_df['Team'].values)
#         fran_weights.insert(len(fran_weights.columns), "Year", feature_df['Year'].values)
    
#         for index, row in results_range.iterrows():
#             # print(row['Year'])
#             # print(index, row)
#             team_1_weight = fran_weights[(fran_weights['Team']==row['Team 1']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
#             # print(team_1_weight)
#             team_2_weight = fran_weights[(fran_weights['Team']==row['Team 2']) & (fran_weights['Year']==row['Year'])]['Scaled Sum'].values
#             # print(team_2_weight)
#             if team_1_weight > team_2_weight:
#                 results_index.append(row['Team 1'])
#             else:
#                 results_index.append(row['Team 2'])
#         temp_df['Winner'] = results_range['Winner']
#         temp_df['Predicted Winner'] = results_index
#         # print(temp_df)
#         comparison_column = np.where(temp_df["Winner"] == temp_df["Predicted Winner"], 1, 0)
#         # print(sum(comparison_column))
#         accuracy = (sum(comparison_column)/temp_df.shape[0])*100
#         # print(performance_dict)
#         # print(len(performance_dict))
#         # print(accuracy, )
#         accuracy_list.append(accuracy)
#         weight_list.append(weights)
#         i+=1
#         pass
#     weights_df['Accuracy'] = accuracy_list
#     weights_df['Weights'] = weight_list
        
    
        
#     return(weights_df)


    


In [74]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

def best_weights(feature_df, results_df, num_iterations=200, n_jobs=4):
    num_features = len(feature_df.columns[2:-1])
    results_range = results_df[results_df['Year'].isin(feature_df['Year'].unique())]
    
    def single_iteration(_):
        weights = np.random.uniform(-1, 1, size=num_features).astype(np.float32)
        fran_weights = feature_df.iloc[:, 2:-3].mul(weights[2:], axis=1)
        fran_weights = fran_weights.mul(feature_df['Strength of Schedule'] * weights[0], axis=0)
        fran_weights = fran_weights.mul(feature_df['Last 10 Rating'] * weights[1], axis=0)
        fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
        fran_weights['Team'] = feature_df['Team'].values
        fran_weights['Year'] = feature_df['Year'].values
        
        results_index = []
        for _, row in results_range.iterrows():
            team_1_weight = fran_weights[(fran_weights['Team'] == row['Team 1']) & 
                                         (fran_weights['Year'] == row['Year'])]['Scaled Sum'].values
            team_2_weight = fran_weights[(fran_weights['Team'] == row['Team 2']) & 
                                         (fran_weights['Year'] == row['Year'])]['Scaled Sum'].values
            results_index.append(row['Team 1'] if team_1_weight > team_2_weight else row['Team 2'])
        
        comparison_column = np.where(results_range['Winner'] == results_index, 1, 0)
        accuracy = (np.sum(comparison_column) / len(comparison_column)) * 100
        return accuracy, weights
    
    results = Parallel(n_jobs=n_jobs)(delayed(single_iteration)(i) for i in tqdm(range(num_iterations)))
    accuracy_list, weight_list = zip(*results)
    
    weights_df = pd.DataFrame({'Accuracy': accuracy_list, 'Weights': weight_list})
    return weights_df

# Example usage:
# feature_df = pd.read_csv('feature_data.csv')
# results_df = pd.read_csv('results_data.csv')
# optimized_weights_df = best_weights(feature_df, results_df)

In [75]:
features_df = pd.read_csv("../data/2007-2025_MBB_Historical-Data.csv")
results_df = pd.read_csv("../data/Bracket_Historical_Data.csv")
# print(results_df, features_df)
weights_df = best_weights(features_df, results_df)
print(weights_df)


















































100%|██████████| 200/200 [01:03<00:00,  3.13it/s]


      Accuracy                                            Weights
0    66.607460  [0.8358988, -0.17095476, -0.078259364, 0.26090...
1    69.715808  [-0.46521777, -0.46532875, 0.96826005, 0.65413...
2    59.325044  [0.059324943, -0.68438643, -0.49136376, 0.5391...
3    69.715808  [0.12179199, 0.33105922, 0.1505765, 0.02593005...
4    31.172291  [0.7283951, -0.73771995, 0.54596555, 0.7909973...
..         ...                                                ...
195  32.149201  [-0.9695793, -0.54342264, 0.013131958, -0.0264...
196  65.186501  [0.087644, -0.12549494, 0.26312542, -0.4481043...
197  33.392540  [0.71865517, -0.5333386, 0.4882787, -0.3811865...
198  48.046181  [0.72703403, 0.628793, -0.014821056, 0.2040344...
199  64.653641  [-0.7591636, 0.3163193, -0.8310586, -0.9360029...

[200 rows x 2 columns]


In [76]:
# features_df = pd.read_csv("../data/2007-2025_MBB_Historical-Data.csv")
# results_df = pd.read_csv("../data/Bracket_Historical_Data.csv")
# # print(results_df, features_df)
# weights_df = best_weights(features_df, results_df)
# print(weights_df)

In [77]:
weights_df = weights_df.sort_values(by=['Accuracy'], ascending=False)

In [78]:
print(weights_df)

      Accuracy                                            Weights
186  70.515098  [0.85276634, 0.76450694, 0.34348753, 0.0748946...
133  70.337478  [0.1720841, 0.9083471, 0.5610719, 0.6562083, -...
191  70.337478  [0.032795757, 0.36991316, 0.92986417, -0.05967...
50   70.159858  [-0.4728612, -0.96747476, -0.19724093, 0.48246...
93   70.159858  [-0.21177976, -0.45977437, -0.48631147, -0.296...
..         ...                                                ...
17   29.928952  [0.6513682, -0.46935028, 0.09750218, 0.1414670...
128  29.928952  [0.8902642, 0.31534868, 0.23546997, 0.01970439...
116  29.751332  [-0.6095716, 0.7887073, 0.91320455, 0.27566037...
189  29.396092  [-0.20755774, -0.6810471, 0.14661212, -0.25314...
46   29.218472  [0.6946765, 0.88744247, -0.9400659, -0.3292428...

[200 rows x 2 columns]


In [79]:
weights = weights_df['Weights'][0]
results_index = 0
# print(features_df)
df = features_df[features_df['Year']==2025]
# print(df)
fran_weights = df.iloc[:, 2:-3].T.apply(lambda x: x * weights[:-2])
fran_weights=fran_weights.T
# fran_weights = fran_weights.multiply(df['Strength of Schedule']*weights[-2], axis=0).multiply(df['Last 10 Rating']*weights[-1], axis=0)
fran_weights['Scaled Sum'] = fran_weights.sum(axis=1)
fran_weights['Team'] = features_df['Team']
fran_weights['Year'] = features_df['Year']
fran_weights = fran_weights.sort_values(by=['Scaled Sum'], ascending=False)
fran_weights.to_csv('Prediction_sum.csv')
